## 实验四
- 静态网页的数据获取
- 动态页面的数据获取

完成实验内容，将此**Jupyter文件以学号姓名.ipynb命名**，如：**2010123456张三.ipynb**。    
当日18点前发送至邮箱1137212597@qq.com。**邮件标题**请注明学号姓名，如：**2010123456张三**。请严格根据题目要求、附件格式及邮件标题提交。    
本次实验成绩纳入平时成绩。

### 一、静态网页爬虫
网址： https://www.1905.com/mdb/film/search/
- 获取评分排名前100的中国电影的电影名称、评分、主演，电影类型存入1.xlsx文件的名称为top100的sheet中。

- 获取电影开国大典（ 网址：https://www.1905.com/mdb/film/2129820/ ）中
  导演：李前宽 ，主演：古月、孙飞虎、黄凯，分别有哪些作品？
  将这些作品的 年份、名称、导演、主演 存入1.xlsx文件的名称为china的sheet中。

- （可选）：基于上一步的数据，根据李前宽、古月、孙飞虎、黄凯的互相合作次数、绘出他们之间的关系图。


# 1.获取评分前100电影数据

In [77]:
from  selenium import webdriver 
import re
import pandas as pd
import time as ti
import numpy as np
import urllib
import os
from pyecharts import options as opts
from pyecharts.charts import Radar
from pyecharts.faker import Collector
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

In [79]:
movie_data=pd.DataFrame(columns={'电影名称':"",'评分':"",'主演':"",'电影类型':""},index=range(120))
movie_datas=pd.DataFrame(columns={'电影名称':"",'评分':"",'主演':"",'电影类型':""},index=range(100))
browser = webdriver.Chrome() 
url=['https://www.1905.com/mdb/film/list/country-China/o1d0p1.html','https://www.1905.com/mdb/film/list/country-China/o1d0p2.html',
    'https://www.1905.com/mdb/film/list/country-China/o1d0p3.html','https://www.1905.com/mdb/film/list/country-China/o1d0p4.html']
for i in range(4):
    browser.get(url[i])
    source=browser.page_source 
    names=re.findall('<p><a target="_blank" title=.*?">(.*?)</a></p>',source,re.S)
    scores=re.findall('<p>.*?评分: <b class="num">(.*?)</b>.*?</p>',source,re.S)
    actor_s=re.findall('<p class="zy">主演：\n(.*?)</p>',source,re.S)
    actors=[]
    for j in actor_s:
        actor=re.findall('<a target="_blank" href="/mdb/star/.*?">(.*?)</a>',j,re.S)
        if(len(actor)!=0):
            a=actor[0]
        else:
            a=""
        for k in actor[1:]:
            a=a+','+k
        actors.append(a)
    type_s=re.findall('<p class="zy">类型：\n(.*?)</p>',source,re.S)
    types=[]
    for j in type_s:
        type_ss=re.findall('<a target="_blank" href="/mdb/film/.*?">(.*?)</a>',j,re.S)
        if(len(type_ss)!=0):
            a=type_ss[0]
        else:
            a=""
        for k in type_ss[1:]:
            a=a+','+k
        types.append(a)
    movie_data["电影名称"][i*30:(i+1)*30]=names
    movie_data["评分"][i*30:(i+1)*30]=scores
    movie_data["主演"][i*30:(i+1)*30]=actors
    movie_data["电影类型"][i*30:(i+1)*30]=types
    ti.sleep(10)
movie_datas=movie_data.iloc[0:100,:]
print(movie_datas)
movie_datas.to_excel("1.xlsx","top100",index=False)
browser.close()

             电影名称   评分                                                 主演  \
0            霸王别姬  9.6  张国荣,巩俐,张丰毅,吕齐,英达　,李春,雷汉,马明威,费振翔,尹治,赵海龙,李丹,蒋雯丽,...   
1            旋风九日  9.5  邓小平,亨利·基辛格,吉米·卡特,兹比格涅夫·布热津斯基,保罗·凯利,凯莉·凡·德赖斯,贝霖...   
2            一代宗师  9.5  梁朝伟,章子怡,张震,宋慧乔,赵本山,小沈阳,梁小龙,王庆祥,张智霖,黎烈弓,张晋,尚铁龙,...   
3            天堂回信  9.4                                           石晨,李丁,肖雄   
4            举起手来  9.4                                  郭达,潘长江,李明,刘小微,胡晓光   
5            骆驼祥子  9.4                                 张丰毅,斯琴高娃,殷新,颜彼得,李唐   
6              茶馆  9.4        于是之,郑榕,蓝天野,英若诚,胡宗温,童超,程中岳,童弟,朱旭,林连昆,黄宗洛,李大千   
7   喜羊羊与灰太狼之兔年顶呱呱  9.3                           祖丽晴,张琳,邓玉婷,梁颖,高全胜,谢维彤,赵娜   
8            小平您好  9.3                                                邓小平   
9            大闹天宫  9.3                               邱岳峰,毕克,富润生,刘广宁,李梓,尚华   
10           杀破狼2  9.2   吴京,托尼·贾,张晋,任达华,古天乐,卢惠光,林嘉华,姜皓文,张弛,恭硕良,寇占文,艾威,蔡瀚亿   
11           爱情天梯  9.2                                          谢兰,王诗槐,格桑   

# 2.获取李前宽、古月、孙飞虎、黄凯作品数据

## 2.1李前宽数据 

In [80]:
def appends(a,b):
    for i in b:
        a.append(i)
    return a

In [81]:
browser = webdriver.Chrome() 
url=['https://www.1905.com/mdb/star/1788928/works/?fr=mdbypk_dh_zp&p=1','https://www.1905.com/mdb/star/1788928/works/?fr=mdbypk_dh_zp&p=2']
movie_datass=pd.DataFrame(columns={'年份':"",'名称':"",'导演':"",'主演':""},index=range(76))
times=[]
names=[]
directs=[]
actors=[]
for i in range(2):
    browser.get(url[i])
    source=browser.page_source
    movie=re.findall('<dl class="clearfloat">\n(.*?)</dl>',source,re.S)
    for j in movie:
        time=re.findall('<dt class="productsTime"><span></span><em>(.*?)</em>',j,re.S)
        name=re.findall('<a href="//www.1905.com/mdb/film/.*?" title=.*? target="_blank">(.*?)</a>',j)
        for kk in range(len(name)):
            appends(times,time)
        appends(names,name)
        direct_s=re.findall('<li class="clearfloat"><i>导演.*?</i>(.*?)</li>',j,re.S)
        for kkk in direct_s:
            direct_ss=re.findall('<a title=".*?" target="_blank" href="/mdb/star/.*?">(.*?)</a>',kkk,re.S)
            if(len(direct_ss)!=0):
                direct=direct_ss[0]
            else:
                direct=""
            for k in direct_ss[1:]:
                direct=direct+','+k
            directs.append(direct)
        actor_s=re.findall('<li class="clearfloat"><i>主演.*?</i>(.*?)</li>',j,re.S)
        for kk in actor_s:
            actor_ss=re.findall('<a title=".*?" target="_blank" href="/mdb/star/.*?">(.*?)</a>',kk,re.S)
            if(len(actor_ss)!=0):
                actor=actor_ss[0]
            else:
                actor=""
            for k in actor_ss[1:]:
                actor=actor+','+k   
            actors.append(actor)
movie_datass["年份"][0:17]=times
movie_datass["名称"][0:17]=names
movie_datass["导演"][0:17]=directs
movie_datass["主演"][0:17]=actors
#movie_datass.to_excel("1.xlsx",sheet_name="china",index=False)
print(movie_datass)
browser.close()

      年份                名称       导演                                       主演
0   2018               韩玉娘  李前宽,肖桂云                                      王冠丽
1   2011                星海  李前宽,肖桂云                              徐若瑄,苏有朋,王成阳
2   2001  "The Korean War"  李前宽,肖桂云  Daniella Kuhn,Richard Lynch,Frank Novak
3   1998              世纪之梦      李前宽                               许还山,严敏求,刘烨
4   1997              旭日惊雷  李前宽,肖桂云                               吴若甫,古月,杜振清
5   1996               红盖头  李前宽,肖桂云                               剧雪,赵岩松,李法曾
6   1995              七七事变  李前宽,肖桂云                              李法曾,吴京安,杜志国
7   1995              金戈铁马  李前宽,肖桂云                               古月,孙海英,丁关宜
8   1993              重庆谈判  李前宽,肖桂云                              董钰刚,胡慧中,李法曾
9   1991              决战之后  李前宽,肖桂云                                陈申生,古月,黄凯
10  1989              开国大典  李前宽,肖桂云                                古月,孙飞虎,黄凯
11  1988               鬼仙沟  肖桂云,李前宽                              刘廷尧,许忠全,栾福仁

## 2.2古月数据 

In [82]:
browser = webdriver.Chrome() 
url=['https://www.1905.com/mdb/star/6939/works/?fr=mdbypk_dh_zp','https://www.1905.com/mdb/star/6939/works/?fr=mdbypk_dh_zp&p=2',
     'https://www.1905.com/mdb/star/6939/works/?fr=mdbypk_dh_zp&p=3']
times=[]
names=[]
directs=[]
actors=[]
for i in range(3):
    browser.get(url[i])
    source=browser.page_source
    movie=re.findall('<dl class="clearfloat">\n(.*?)</dl>',source,re.S)
    for j in movie:
        time=re.findall('<dt class="productsTime"><span></span><em>(.*?)</em>',j,re.S)
        name=re.findall('<a href="//www.1905.com/mdb/film/.*?" title=.*? target="_blank">(.*?)</a>',j)
        for kk in range(len(name)):
            appends(times,time)
        appends(names,name)
        movies=re.findall('<dd class="clearfix.*?">(.*?)</dd>',j,re.S)
        for kk in movies:
            direct_s=re.findall('<li class="clearfloat"><i>导演.*?</i>(.*?)</li>',kk,re.S)
            if(len(direct_s)==0):
                directs.append("")
            for kkk in direct_s:
                direct_ss=re.findall('<a title=".*?" target="_blank" href="/mdb/star/.*?">(.*?)</a>',kkk,re.S)
                if(len(direct_ss)!=0):
                    direct=direct_ss[0]
                else:
                    direct=""
                for k in direct_ss[1:]:
                    direct=direct+','+k
                directs.append(direct)
        actor_s=re.findall('<li class="clearfloat"><i>主演.*?</i>(.*?)</li>',j,re.S)
        if(len(actor_s)==0):
            actors.append("")
        for kk in actor_s:
            actor_ss=re.findall('<a title=".*?" target="_blank" href="/mdb/star/.*?">(.*?)</a>',kk,re.S)
            if(len(actor_ss)!=0):
                actor=actor_ss[0]
            else:
                actor=""
            for k in actor_ss[1:]:
                actor=actor+','+k   
            actors.append(actor)
movie_datass["年份"][17:40]=times
movie_datass["名称"][17:40]=names
movie_datass["导演"][17:40]=directs
movie_datass["主演"][17:40]=actors
#movie_datass.to_excel("1.xlsx",sheet_name="china",index=False)
print(movie_datass)
browser.close()

      年份                名称       导演                                       主演
0   2018               韩玉娘  李前宽,肖桂云                                      王冠丽
1   2011                星海  李前宽,肖桂云                              徐若瑄,苏有朋,王成阳
2   2001  "The Korean War"  李前宽,肖桂云  Daniella Kuhn,Richard Lynch,Frank Novak
3   1998              世纪之梦      李前宽                               许还山,严敏求,刘烨
4   1997              旭日惊雷  李前宽,肖桂云                               吴若甫,古月,杜振清
5   1996               红盖头  李前宽,肖桂云                               剧雪,赵岩松,李法曾
6   1995              七七事变  李前宽,肖桂云                              李法曾,吴京安,杜志国
7   1995              金戈铁马  李前宽,肖桂云                               古月,孙海英,丁关宜
8   1993              重庆谈判  李前宽,肖桂云                              董钰刚,胡慧中,李法曾
9   1991              决战之后  李前宽,肖桂云                                陈申生,古月,黄凯
10  1989              开国大典  李前宽,肖桂云                                古月,孙飞虎,黄凯
11  1988               鬼仙沟  肖桂云,李前宽                              刘廷尧,许忠全,栾福仁

## 2.3孙飞虎数据 

In [83]:
browser = webdriver.Chrome() 
url=['https://www.1905.com/mdb/star/8539/works/?fr=mdbypk_dh_zp&p=1','https://www.1905.com/mdb/star/8539/works/?fr=mdbypk_dh_zp&p=2',
     'https://www.1905.com/mdb/star/8539/works/?fr=mdbypk_dh_zp&p=3']
times=[]
names=[]
directs=[]
actors=[]
for i in range(3):
    browser.get(url[i])
    source=browser.page_source
    movie=re.findall('<dl class="clearfloat">\n(.*?)</dl>',source,re.S)
    for j in movie:
        time=re.findall('<dt class="productsTime"><span></span><em>(.*?)</em>',j,re.S)
        name=re.findall('<a href="//www.1905.com/mdb/film/.*?" title=.*? target="_blank">(.*?)</a>',j)
        for kk in range(len(name)):
            appends(times,time)
        appends(names,name)
        movies=re.findall('<dd class="clearfix.*?">(.*?)</dd>',j,re.S)
        for kk in movies:
            direct_s=re.findall('<li class="clearfloat"><i>导演.*?</i>(.*?)</li>',kk,re.S)
            if(len(direct_s)==0):
                directs.append("")
            for kkk in direct_s:
                direct_ss=re.findall('<a title=".*?" target="_blank" href="/mdb/star/.*?">(.*?)</a>',kkk,re.S)
                if(len(direct_ss)!=0):
                    direct=direct_ss[0]
                else:
                    direct=""
                for k in direct_ss[1:]:
                    direct=direct+','+k
                directs.append(direct)
        actor_s=re.findall('<li class="clearfloat"><i>主演.*?</i>(.*?)</li>',j,re.S)
        if(len(actor_s)==0):
            actors.append("")
        for kk in actor_s:
            actor_ss=re.findall('<a title=".*?" target="_blank" href="/mdb/star/.*?">(.*?)</a>',kk,re.S)
            if(len(actor_ss)!=0):
                actor=actor_ss[0]
            else:
                actor=""
            for k in actor_ss[1:]:
                actor=actor+','+k   
            actors.append(actor)
movie_datass["年份"][40:67]=times
movie_datass["名称"][40:67]=names
movie_datass["导演"][40:67]=directs
movie_datass["主演"][40:67]=actors
#movie_datass.to_excel("1.xlsx",sheet_name="china",index=False)
print(movie_datass)
browser.close()

      年份                名称       导演                                       主演
0   2018               韩玉娘  李前宽,肖桂云                                      王冠丽
1   2011                星海  李前宽,肖桂云                              徐若瑄,苏有朋,王成阳
2   2001  "The Korean War"  李前宽,肖桂云  Daniella Kuhn,Richard Lynch,Frank Novak
3   1998              世纪之梦      李前宽                               许还山,严敏求,刘烨
4   1997              旭日惊雷  李前宽,肖桂云                               吴若甫,古月,杜振清
5   1996               红盖头  李前宽,肖桂云                               剧雪,赵岩松,李法曾
6   1995              七七事变  李前宽,肖桂云                              李法曾,吴京安,杜志国
7   1995              金戈铁马  李前宽,肖桂云                               古月,孙海英,丁关宜
8   1993              重庆谈判  李前宽,肖桂云                              董钰刚,胡慧中,李法曾
9   1991              决战之后  李前宽,肖桂云                                陈申生,古月,黄凯
10  1989              开国大典  李前宽,肖桂云                                古月,孙飞虎,黄凯
11  1988               鬼仙沟  肖桂云,李前宽                              刘廷尧,许忠全,栾福仁

## 2.4黄凯数据 

In [84]:
browser = webdriver.Chrome() 
url=['https://www.1905.com/mdb/star/1975100/works/?fr=mdbypk_dh_zp']
times=[]
names=[]
directs=[]
actors=[]
for i in range(1):
    browser.get(url[i])
    source=browser.page_source
    movie=re.findall('<dl class="clearfloat">\n(.*?)</dl>',source,re.S)
    for j in movie:
        time=re.findall('<dt class="productsTime"><span></span><em>(.*?)</em>',j,re.S)
        name=re.findall('<a href="//www.1905.com/mdb/film/.*?" title=.*? target="_blank">(.*?)</a>',j)
        for kk in range(len(name)):
            appends(times,time)
        appends(names,name)
        movies=re.findall('<dd class="clearfix.*?">(.*?)</dd>',j,re.S)
        for kk in movies:
            direct_s=re.findall('<li class="clearfloat"><i>导演.*?</i>(.*?)</li>',kk,re.S)
            if(len(direct_s)==0):
                directs.append("")
            for kkk in direct_s:
                direct_ss=re.findall('<a title=".*?" target="_blank" href="/mdb/star/.*?">(.*?)</a>',kkk,re.S)
                if(len(direct_ss)!=0):
                    direct=direct_ss[0]
                else:
                    direct=""
                for k in direct_ss[1:]:
                    direct=direct+','+k
                directs.append(direct)
        actor_s=re.findall('<li class="clearfloat"><i>主演.*?</i>(.*?)</li>',j,re.S)
        if(len(actor_s)==0):
            actors.append("")
        for kk in actor_s:
            actor_ss=re.findall('<a title=".*?" target="_blank" href="/mdb/star/.*?">(.*?)</a>',kk,re.S)
            if(len(actor_ss)!=0):
                actor=actor_ss[0]
            else:
                actor=""
            for k in actor_ss[1:]:
                actor=actor+','+k   
            actors.append(actor)
movie_datass["年份"][67:]=times
movie_datass["名称"][67:]=names
movie_datass["导演"][67:]=directs
movie_datass["主演"][67:]=actors
movie_datass.to_excel("1.xlsx",sheet_name="china",index=False)
print(movie_datass)
browser.close()

      年份                名称       导演                                       主演
0   2018               韩玉娘  李前宽,肖桂云                                      王冠丽
1   2011                星海  李前宽,肖桂云                              徐若瑄,苏有朋,王成阳
2   2001  "The Korean War"  李前宽,肖桂云  Daniella Kuhn,Richard Lynch,Frank Novak
3   1998              世纪之梦      李前宽                               许还山,严敏求,刘烨
4   1997              旭日惊雷  李前宽,肖桂云                               吴若甫,古月,杜振清
5   1996               红盖头  李前宽,肖桂云                               剧雪,赵岩松,李法曾
6   1995              七七事变  李前宽,肖桂云                              李法曾,吴京安,杜志国
7   1995              金戈铁马  李前宽,肖桂云                               古月,孙海英,丁关宜
8   1993              重庆谈判  李前宽,肖桂云                              董钰刚,胡慧中,李法曾
9   1991              决战之后  李前宽,肖桂云                                陈申生,古月,黄凯
10  1989              开国大典  李前宽,肖桂云                                古月,孙飞虎,黄凯
11  1988               鬼仙沟  肖桂云,李前宽                              刘廷尧,许忠全,栾福仁

# 3.分析 李前宽，古月，孙飞虎，黄凯的关系

In [85]:
relation=pd.DataFrame(columns={'李前宽':"",'古月':"",'孙飞虎':"",'黄凯':""},index={"李前宽","古月","孙飞虎","黄凯"})
strr=["李前宽","古月","孙飞虎","黄凯"]
start=[0,17,40,67]
end=[17,40,67,76]
def rela():
    back=np.array([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])
    for n in range(4):
        for i in range(start[n],end[n]):
            for j in range(4):
                if(j!=n):
                    if(strr[j] in movie_datass.loc[i,"导演"] or strr[j] in movie_datass.loc[i,"主演"]):
                        back[n,j]=back[n,j]+1
    for i in range(4):
        for j in range(i,4):
            if(i!=j):
                if(back[i,j]<=back[j,i]):
                    back[i,j]=back[j,i]
                else:
                    back[j,i]=back[i,j]  
    return back
ll=rela()
radar = Radar()
# //由于雷达图传入的数据一般是多维数据，需要做一下处理
# # //设置column的最大值，为了雷达图更为直观，这里的月份最大值设置有所不同
schema = [ 
    {"name": "李前宽", "max": 5, "min": 0},{"name": "古月", "max": 5, "min": 0},
    {"name": "孙飞虎", "max": 5, "min": 0},{"name": "黄凯", "max": 5, "min": 0}]
# //传入坐标
radar.add_schema(schema,shape="polygon")
radar.add("李前宽",ll[0:1].tolist(),color="#f9713c")
radar.add("古月",ll[1:2].tolist(),color="#4169E1")
radar.add("孙飞虎",ll[2:3].tolist(),color="#00BFFF")
radar.add("黄凯",ll[3:4].tolist(),color="#3CB371")
# //一般默认为同一种颜色，这里为了便于区分，需要设置item的颜色
radar.render_notebook()

### 二、动态网页爬虫
网址： https://image.so.com/  ，以关键词搜索python 。   
- 获取50张大尺寸的图片标题、图片链接，图片文件（下载图片并保存在当前路径下的data文件夹）。 (提示：大尺寸图片在搜索结果右上可筛选)   
- 将包含本地绝对路径的文件名、图片标题(去除标题中的HTML标签)、图片链接 存入1.xlsx文件的名称为pic的sheet中。

# 4.通过动态网页动作捕获50张大尺寸图片标题+链接，并下载

In [89]:
from selenium.webdriver.support.ui import Select
browser = webdriver.Chrome() 
browser.get("https://image.so.com/")
input = browser.find_element_by_id('search_kw')
input.send_keys('python')
button = browser.find_element_by_id('hd_search').find_element_by_tag_name('form').find_element_by_tag_name('button')
button.click()
t = browser.find_element_by_id('filterBox').find_element_by_tag_name('ul').find_element_by_class_name('size')
t.click()
ti.sleep(5)
button1 = browser.find_element_by_id('filterBox').find_element_by_tag_name('ul').find_element_by_class_name('size').find_element_by_tag_name('dl').find_element_by_tag_name('dd')
button1.click()
browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
ti.sleep(3)
browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
source=browser.page_source
cell=re.findall('<li class="cell" data-id=.*?>(.*?)</li>',source,re.S)
titles=[]
urls=[]
i=0
num=0
while num<50:
    title=re.findall('<img style=".*?" src=".*?" alt="(.*?)" />',cell[i],re.S)
    url=re.findall('<img style=".*?" src="(.*?)" alt=".*?" />',cell[i],re.S)
    if(len(url)!=0):
        titles.append(title[0])
        urls.append(url[0])
        num=num+1
    i=i+1
if not os.path.exists("data"):
    os.makedirs("data") 
for j in range(50):
    urllib.request.urlretrieve(urls[j], 'data\\'+str(j+1)+'.jpg')
browser.close()

# 5.将图片绝对路径(我的)，标题，链接存入excel中

In [90]:
picture_data=pd.DataFrame(columns={'文件名':"",'图片标题':"",'图片链接':""},index=range(50))
picture_data["图片标题"]=titles
picture_data["图片链接"]=urls
for i in range(1,51):
    picture_data.iloc[i-1,0]="C:\\Users\\周凤宇\\Desktop\\学习资料\\大四上学期\\Python\\第四次实验\\data\\"+str(i)+'.jpg'
print(picture_data)
picture_data.to_excel("1.xlsx",sheet_name="pic",index=False)

                                                  文件名  \
0   C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
1   C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
2   C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
3   C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
4   C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
5   C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
6   C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
7   C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
8   C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
9   C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
10  C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
11  C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
12  C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
13  C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
14  C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
15  C:\Users\周凤宇\Desktop\学习资料\大四上学期\Python\第四次实验\d...   
16  C:\Users\周凤宇\Desktop\学习资料\大

## 注意：我注意到每次写入不同的sheet会产生覆盖效果，我下面的实现可以写入电影数据和图片数据并不覆盖 

In [92]:
writer = pd.ExcelWriter("1.xlsx")
movie_datas.to_excel(writer,"top100",index=False)
movie_datass.to_excel(writer,"china",index=False)
picture_data.to_excel(writer,"pic",index=False)
writer.save()
writer.close()